In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense

# Load the dataset
train = pd.read_csv('../Datasets/NB.csv')
train['Fault'] = 0
test = pd.read_csv('../Datasets/IR - 7.csv')
test['Fault'] = 1
dataset = train.append(test)

# Prepare the input and output variables
X = dataset.iloc[:, 0:2].values
y = dataset.iloc[:, 2]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Standardize the input variables
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Define the neural network model
classifier = Sequential()
classifier.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'relu', input_dim = 2))
classifier.add(Dense(units = 2, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compile the model
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Train the model
model = classifier.fit(X_train, y_train, batch_size = 1000, epochs = 100, shuffle = False)

# Predict the output using the trained model
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Evaluate the performance of the model
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, columns=['Normal', 'Inner Race (0.007")'], index=['Normal', 'Inner Race (0.007")'])
plt.figure(figsize=(10,4))
sn.set(font_scale=1.4)
sn.heatmap(cm_df, annot=True, fmt='g')
plt.title('Confusion matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

evaluation = classifier.evaluate(X_test, y_test)
print("Accuracy: {:.2f}%".format(evaluation[1]*100))

cr = classification_report(y_test, y_pred, target_names=['Normal', 'Inner Race (0.007")'])
print(cr)

# Count the number of anomalies in the predictions
normal, IR = cm
anomalies = normal[1] + IR[0]
print("Anomalies detected: {}".format(anomalies))
